In [ ]:
# Standard libraries of Python

from time import gmtime

# Dependencies
import pandas as pd
import requests
import re

from pandas import DataFrame
from bs4 import BeautifulSoup,Tag
from dataclasses import dataclass
from selectolax.parser import HTMLParser
from tqdm import tqdm

# Librarys
from backend.database.src.src import Pokemon, Mega_Pokemon
from backend.database.src.parse import process_hidden_ability,process_ability,process_form_ability

In [ ]:
#Class for selectolax scrap
# @dataclass
# class Pokemon:
#     name: str
#     gender: str
#     types: list

In [ ]:
#Selecto scrap
pokedex = []
url = 'https://www.serebii.net/pokedex-sm/{}.shtml'.format(str(1).zfill(3))
html = requests.get(url).text
tree = HTMLParser(html)
pokemon = Pokemon(name=tree.css_first('#content > main > div > div > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(1)').text(),
            gender=tree.css_first('#content > main > div > div > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(4)').text(),
            types=list(map(lambda x: x.attributes['alt'], tree.css('#content > main > div > div > table:nth-child(5) > tbody > tr:nth-child(2) > td.cen >> img'))))
pokedex.append(pokemon)

pd.DataFrame(pokedex)

In [ ]:
#Examble of use with bs4
html_text = requests.get(f'https://www.serebii.net/pokedex-sm/006.shtml').text
soup = BeautifulSoup(html_text, 'html.parser')
all_divs = soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[0].find_all('td', {'class': 'footype'})

In [ ]:
#Class test
x = Pokemon(7,6)

all_divs = x.soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[0].find_all('td', {'class': 'footype'})

In [ ]:
#Abilities
ala = foo_info.find_all('b')

abilities = {'ability': [], 'hidden_ability': []}
form_abilities = {'ability': [], 'hidden_ability': []}
skip_next_flag = False
form_abilities_flag = False

for tag in ala:
    if skip_next_flag:
        skip_next_flag = False
        continue

    if "Hidden Ability" in tag.text:
        skip_next_flag = process_hidden_ability(tag, abilities, form_abilities, skip_next_flag, form_abilities_flag)
    else:
        form_abilities_flag = process_form_ability(tag, form_abilities_flag)
        process_ability(tag, abilities, form_abilities, form_abilities_flag)

form_abilities_flag = False

In [ ]:
#Weakness
types = []

for td_tag in foo_info[0:18]:
    a_tag = td_tag.find('img')
    if a_tag:
        type_text = a_tag['alt']
        types.append(type_text)

s = 'Attacking Move Type: ','-type'
for string in s:
    types = list(map(lambda x: x.replace(string,''),types))

print(types)

filtered = [tag for tag in foo_info[18:36] if '*' in tag.get_text(strip=True)]
filtered_t = [i.text for i in filtered]
val = list(map(lambda x: int(x.replace('*','')),filtered_t))

In [ ]:
#Elemental Types
tp = foo_info[0]
types = []
for tag in tp:
    print(f'tag a añadir:{tag}')
    a_tag = tag.find('img')
    if a_tag and not isinstance(a_tag,Tag):
        continue
    else:
        type_text = a_tag['alt']
        types.append(type_text)

s = 'Attacking Move Type: ','-type'
for string in s:
    types = list(map(lambda x: x.replace(string,''),types))

In [ ]:
#Get al tables from the main center table
tables = all_divs[0].find_all('table', {'class': 'dextable'})

#Search for the line that contains <td class="fooevo" colspan="6">
mega_evolution_position = -1
counter = 0
for i, table in enumerate(tables):
    if table.find('td', {'class': 'fooevo', 'colspan': '6'}):
        mega_evolution_position = i
        print(i)
        print(table)
        break #Simulates return
    else:
        print('Does not have Mega')

In [ ]:
#Mega Evolution Types
t = tables[19].find('td', class_='cen')

ty_val = []

for i in t:
    n = i.find('img')
    if n and not isinstance(n,Tag):
        continue
    n = n['src']
    for element in x.elemental_types:
        ty = element.lower()
        if ty in n:
            ty_val.append(element)

print(ty_val)

In [ ]:
#Mega Evolution Weakness
w = []

if ty_val == x._elements:
    pass
else:
    weak_location = tables[22].find('td', string=re.compile("Base Stats - Total.*")).find_next_siblings('td')
    filtered_types = list(filter(lambda x: '*' in x.text, weak_location.find_all('td', {'class': 'footype'})))
    filtered = [tag.text for tag in filtered_types if '*' in tag.get_text(strip=True)]

    val = list(map(lambda x: x.replace('*',''),filtered_t))

    for i in val:
        try:
            w.append(int(i))
        except ValueError:
            w.append(float(i))

    mega_weak = dict(zip(x.elemental_types,w))